## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [2]:
# File location and type
file_location = "/FileStore/tables/Lockdown_Join_with_normaliztion_factor-ed399.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

##display(df)

In [3]:
from pyspark import SparkContext

#sc = SparkContext()

In [4]:
df.printSchema()


root
-- date: string (nullable = true)
-- state: string (nullable = true)
-- fips: integer (nullable = true)
-- cases: integer (nullable = true)
-- deaths: integer (nullable = true)
-- Date_Start_of_Lockdown: string (nullable = true)
-- Date_End_of_Lockdown: string (nullable = true)
-- Lockdown_Length: string (nullable = true)
-- Noramliztion factor (per 1,000): double (nullable = true)

In [5]:
df.head(5)

Out[4]: [Row(date='1/21/2020', state='Washington', fips=53, cases=1, deaths=0, Date_Start_of_Lockdown='23-Mar', Date_End_of_Lockdown='4-May', Lockdown_Length='42', Noramliztion factor (per 1,000)=0.00013132),
 Row(date='1/22/2020', state='Washington', fips=53, cases=1, deaths=0, Date_Start_of_Lockdown='23-Mar', Date_End_of_Lockdown='4-May', Lockdown_Length='42', Noramliztion factor (per 1,000)=0.00013132),
 Row(date='1/23/2020', state='Washington', fips=53, cases=1, deaths=0, Date_Start_of_Lockdown='23-Mar', Date_End_of_Lockdown='4-May', Lockdown_Length='42', Noramliztion factor (per 1,000)=0.00013132),
 Row(date='1/24/2020', state='Illinois', fips=17, cases=1, deaths=0, Date_Start_of_Lockdown='21-Mar', Date_End_of_Lockdown='30-May', Lockdown_Length='70', Noramliztion factor (per 1,000)=7.892e-05),
 Row(date='1/24/2020', state='Washington', fips=53, cases=1, deaths=0, Date_Start_of_Lockdown='23-Mar', Date_End_of_Lockdown='4-May', Lockdown_Length='42', Noramliztion factor (per 1,000)=0.00013132)]

In [6]:
df.show(2,truncate= True)

+---------+----------+----+-----+------+----------------------+--------------------+---------------+-------------------------------+
 date| state|fips|cases|deaths|Date_Start_of_Lockdown|Date_End_of_Lockdown|Lockdown_Length|Noramliztion factor (per 1,000)|
+---------+----------+----+-----+------+----------------------+--------------------+---------------+-------------------------------+
1/21/2020|Washington| 53| 1| 0| 23-Mar| 4-May| 42| 1.3132E-4|
1/22/2020|Washington| 53| 1| 0| 23-Mar| 4-May| 42| 1.3132E-4|
+---------+----------+----+-----+------+----------------------+--------------------+---------------+-------------------------------+
only showing top 2 rows

In [7]:
df.count()

Out[6]: 3576

In [8]:
len(df.columns), df.columns

Out[7]: (9,
 ['date',
 'state',
 'fips',
 'cases',
 'deaths',
 'Date_Start_of_Lockdown',
 'Date_End_of_Lockdown',
 'Lockdown_Length',
 'Noramliztion factor (per 1,000)'])

In [9]:
df.describe().show()

+-------+---------+-------+------------------+-----------------+------------------+----------------------+--------------------+------------------+-------------------------------+
summary| date| state| fips| cases| deaths|Date_Start_of_Lockdown|Date_End_of_Lockdown| Lockdown_Length|Noramliztion factor (per 1,000)|
+-------+---------+-------+------------------+-----------------+------------------+----------------------+--------------------+------------------+-------------------------------+
 count| 3576| 3576| 3576| 3576| 3576| 3576| 3576| 3576| 3576|
 mean| null| null|29.359619686800894| 9531.66023489933| 504.6353467561521| null| null|44.007933509633546| 3.778021448545859E-4|
 stddev| null| null|15.908018201525818|30365.27754584857|2149.7865620836146| null| null|13.497470906072694| 4.093965141874381E-4|
 min|1/21/2020|Alabama| 1| 1| 0| 1-Apr| 1-May| 24| 2.531E-5|
 max| 5/9/2020|Wyoming| 56| 342267| 27003| N/A| N/A| N/A| 0.00172783|
+-------+---------+-------+------------------+-----------------+------------------+----------------------+--------------------+------------------+-------------------------------+

In [10]:
df.describe('cases').show()

+-------+-----------------+
summary| cases|
+-------+-----------------+
 count| 3576|
 mean| 9531.66023489933|
 stddev|30365.27754584857|
 min| 1|
 max| 342267|
+-------+-----------------+

In [11]:
df.select('fips','cases').show(5)

+----+-----+
fips|cases|
+----+-----+
 53| 1|
 53| 1|
 53| 1|
 17| 1|
 53| 1|
+----+-----+
only showing top 5 rows

In [12]:
df.select('fips','cases').distinct().count()

Out[11]: 3187

In [13]:
#df.crosstab('state', 'cases').show()

In [14]:
#df.crosstab('state', 'cases').dropDuplicates().show()

In [15]:
df.dropna().count()

Out[14]: 3576

In [16]:
df.groupby('state').agg({'cases': 'mean'}).show()

+-------------+------------------+
 state| avg(cases)|
+-------------+------------------+
 Utah|1942.4545454545455|
 Hawaii|346.85074626865674|
 Minnesota| 2467.791044776119|
 Ohio| 8406.296875|
 Oregon|1178.9054054054054|
 Arkansas| 1572.532258064516|
 Texas|10063.977777777778|
 North Dakota|486.19354838709677|
 Pennsylvania|21706.194029850747|
 Connecticut|12695.923076923076|
 Nebraska|1436.2941176470588|
 Vermont|499.92424242424244|
 Nevada| 2471.294117647059|
 Washington| 4966.732142857143|
 Illinois|14557.853211009175|
 Oklahoma|1751.3283582089553|
 Delaware| 2133.983870967742|
 Alaska|215.78688524590163|
 New Mexico|1594.1935483870968|
West Virginia| 645.3928571428571|
+-------------+------------------+
only showing top 20 rows

In [17]:
df.groupby('cases').count().show()

+-----+-----+
cases|count|
+-----+-----+
 6620| 1|
 1580| 2|
 463| 1|
25591| 1|
 1342| 1|
 3175| 1|
15846| 1|
 2387| 1|
 392| 2|
 4161| 1|
 623| 2|
 1721| 2|
 540| 1|
 1507| 1|
 4929| 1|
 1025| 1|
23607| 1|
26767| 1|
42348| 1|
 897| 1|
+-----+-----+
only showing top 20 rows

In [18]:
#df.select('date').map(lambda x:(x,1)).take(5)

In [19]:
df.orderBy(df.cases.desc()).show(5)

+---------+--------+----+------+------+----------------------+--------------------+---------------+-------------------------------+
 date| state|fips| cases|deaths|Date_Start_of_Lockdown|Date_End_of_Lockdown|Lockdown_Length|Noramliztion factor (per 1,000)|
+---------+--------+----+------+------+----------------------+--------------------+---------------+-------------------------------+
5/11/2020|New York| 36|342267| 27003| 22-Mar| 15-May| 54| 5.14E-5|
5/10/2020|New York| 36|340657| 26798| 22-Mar| 15-May| 54| 5.14E-5|
 5/9/2020|New York| 36|338519| 26584| 22-Mar| 15-May| 54| 5.14E-5|
 5/8/2020|New York| 36|335804| 26358| 22-Mar| 15-May| 54| 5.14E-5|
 5/7/2020|New York| 36|332931| 26206| 22-Mar| 15-May| 54| 5.14E-5|
+---------+--------+----+------+------+----------------------+--------------------+---------------+-------------------------------+
only showing top 5 rows

In [20]:
%fs ls

path,name,size
dbfs:/FileStore/,FileStore/,0
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-results/,databricks-results/,0
dbfs:/tmp/,tmp/,0


In [21]:
%fs ls dbfs:/databricks-datasets

path,name,size
dbfs:/databricks-datasets/,databricks-datasets/,0
dbfs:/databricks-datasets/COVID/,COVID/,0
dbfs:/databricks-datasets/README.md,README.md,976
dbfs:/databricks-datasets/Rdatasets/,Rdatasets/,0
dbfs:/databricks-datasets/SPARK_README.md,SPARK_README.md,3359
dbfs:/databricks-datasets/adult/,adult/,0
dbfs:/databricks-datasets/airlines/,airlines/,0
dbfs:/databricks-datasets/amazon/,amazon/,0
dbfs:/databricks-datasets/asa/,asa/,0
dbfs:/databricks-datasets/atlas_higgs/,atlas_higgs/,0
